In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
from statsmodels.tsa.seasonal import seasonal_decompose

sns.set(style="whitegrid")

# Folder paths
input_folder = r'C:\Users\jahna\OneDrive\Documents\TIME_SERIES_STOCK_FORECASTING\PREPROCESSING\CLEAN_DATA_FINAL'
plot_output_folder = r'C:\Users\jahna\OneDrive\Documents\TIME_SERIES_STOCK_FORECASTING\VISUALIZATION\plots'
os.makedirs(plot_output_folder, exist_ok=True)

# Visualization function
def generate_visuals(file_path, output_folder):
    df = pd.read_csv(file_path, parse_dates=['date'])
    df = df.sort_values('date')

    stock_name = os.path.splitext(os.path.basename(file_path))[0]

    # --------- Matplotlib: Time Series Line Plot ---------
    plt.figure(figsize=(12, 5))
    plt.plot(df['date'], df['close'], label='Close Price', color='blue')
    plt.title(f"{stock_name} - Close Price Over Time")
    plt.xlabel("Date")
    plt.ylabel("Close Price")
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"{stock_name}_lineplot.png"))
    plt.close()

    # --------- Seaborn: Rolling Averages Plot ---------
    plt.figure(figsize=(12, 5))
    sns.lineplot(x='date', y='close', data=df, label='Close')
    if 'rolling_5_close' in df.columns:
        sns.lineplot(x='date', y='rolling_5_close', data=df, label='5-day MA')
    if 'rolling_20_close' in df.columns:
        sns.lineplot(x='date', y='rolling_20_close', data=df, label='20-day MA')
    plt.title(f"{stock_name} - Rolling Averages")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"{stock_name}_rolling_avg.png"))
    plt.close()

    # --------- Plotly: Interactive Time Series ---------
    fig = px.line(df, x='date', y='close', title=f"{stock_name} - Interactive Close Price")
    fig.write_html(os.path.join(output_folder, f"{stock_name}_interactive.html"))

    # --------- Seasonal Decomposition ---------
    try:
        df.set_index('date', inplace=True)
        df = df.asfreq('D')  # daily frequency for decomposition
        df['close'] = df['close'].interpolate()  # fill gaps

        decomposition = seasonal_decompose(df['close'], model='multiplicative', period=30)
        fig = decomposition.plot()
        fig.set_size_inches(12, 8)
        plt.suptitle(f"{stock_name} - Seasonal Decomposition", fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f"{stock_name}_decomposition.png"))
        plt.close()
    except Exception as e:
        print(f"⚠️ Could not decompose {stock_name}: {e}")

# --------- Loop Through All CSVs ---------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        try:
            generate_visuals(file_path, plot_output_folder)
            print(f"✅ Plots generated for: {file}")
        except Exception as e:
            print(f"❌ Failed to plot {file}: {e}")

print("\n📊 All plots saved in:", plot_output_folder)
